In [1]:
import pickle
from model_arch.tokenizer import load_tokenizer
import torch
import sys
# from pathlib import Path

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

from model_arch import transformer
sys.modules['transformer'] = transformer

In [3]:

best_model_fp = 'models/final/model_comedies_only_df1000.pkl'
# best_model_fp = 'models/results/final_model_df1000.pkl'
# best_model_fp = 'models/results/model_df2000_best_epoch_19560_min_val_loss_0.028599999845027924.pkl'
# final_model_fp = 'models/final_model_df2000.pkl'
with open(best_model_fp, 'rb') as handle:
    best_model = pickle.load(handle)
    
# with open(final_model_fp, 'rb') as handle:
#     final_model = pickle.load(handle)

In [4]:
for param in best_model.parameters():
    param.requires_grad = False
    
# for param in final_model.parameters():
#     param.requires_grad = False

# Comparing diffusion models (by # steps)

In [218]:
from model_arch.gaussian_diffusion import get_named_beta_schedule, SpacedDiffusion
from model_arch.tokenizer import load_tokenizer
from transformers import set_seed
from model_arch.sampling import *

In [219]:
noise_schedule='sqrt'
predict_xstart=True
rescale_timesteps=True
regular_data_dir='data'
seed=24666
config_name='bert-base-uncased'

set_seed(seed)
tokenizer = load_tokenizer('shakespeare', config_name)

shakespeare


In [220]:
diffusion_100 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 100),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

diffusion_500 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 500),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

diffusion_1000 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 1000),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

diffusion_2000 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 2000),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

### With 100 diffusion steps

In [42]:
word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_100, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128, 
                                                           show_intermediate_results=False)

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ["My lord, 'tis the pondering of life's meaning that doth occupy my thoughts most gravely.", 'call her forth to me.'] ['', '']
RAM used: 2602.65 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 7
})
RAM used: 2602.65 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/7 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 7
})
### tokenized_datasets...example [2, 105, 203, 10, 9, 334, 78, 14472, 102, 94, 451, 9, 41, 2679, 110, 457, 19200, 105, 1079, 367, 23844, 12, 3]
RAM used: 2602.72 MB


merge and mask:   0%|          | 0/7 [00:00<?, ? examples/s]

RAM used: 2602.72 MB


padding:   0%|          | 0/7 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 7
}) padded dataset
RAM used: 2602.72 MB
RAM used: 2602.72 MB
### End of reading iteration...


  0%|          | 0/100 [00:00<?, ?it/s]

In [43]:
word_lst_source, word_lst_recover

(["[CLS] my lord,'tis the pondering of life's meaning that doth occupy my thoughts most gravely. [SEP] [SEP]",
  '[CLS] call her forth to me. [SEP] [SEP]',
  '[CLS] what is your will? [SEP] [SEP]',
  '[CLS] hark, fair sarah! [SEP] [SEP]',
  "[CLS] what dost thou have planned for the morrow's reprieve? [SEP] [SEP]",
  '[CLS] to change a master. o, my fortunes have corrupted honest men! dispatch. enobarbus! [SEP] [SEP]',
  '[CLS] away! [SEP] [SEP]'],
 ['[CLS] them thing his air and - for to thee again, in call there he the since,, [SEP]',
  '[CLS], i this your you can much thou aside must, it,, [PAD] [PAD] [SEP]',
  "[CLS] so [SEP]'ll??",
  '[CLS] tis my, give -, [PAD] [PAD] [PAD]. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [SEP]',
  '[CLS] as noise,, good. whither then and? [SEP]',
  '[CLS] less noise, ho [SEP]',
  '[CLS] been,!, find, earth? [PAD] [SEP]'])

### With 500 diffusion steps

In [171]:
inter_steps=[0.1, 0.3, 0.4, 0.5, 0.7]
word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_500, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128,
                                                           inter_steps=inter_steps)

num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.


############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ['No, sir, I live by the church.', 'Call her forth to me'] ['', '']
RAM used: 3910.50 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 2
})
RAM used: 3910.50 MB


Running tokenizer on dataset (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 2
})
### tokenized_datasets...example [2, 95, 10, 220, 10, 31, 615, 193, 78, 1975, 12, 3]
RAM used: 3910.55 MB


merge and mask:   0%|          | 0/2 [00:00<?, ? examples/s]

RAM used: 3910.60 MB


padding:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 2
}) padded dataset
RAM used: 3910.60 MB
RAM used: 3910.65 MB
### End of reading iteration...


  0%|          | 0/500 [00:00<?, ?it/s]

In [172]:
word_lst_source, word_lst_recover

(['[CLS] no, sir, i live by the church. [SEP] [SEP]',
  '[CLS] call her forth to me [SEP] [SEP]'],
 ['[CLS] what, you? [SEP]', '[CLS] i is thou bed, to my will to thee,. [SEP]'])

In [14]:
for i in range(len(inter_steps)):
    print(f"======= {inter_steps[i]*100}% generating... =======\n")
    print(inter_lst_recover[i][0], '\n') 

======= 10.0% generating... =======

shorten how herdsmen a adher fatally unfaithdling freedoms criedst thunderer unconst prevailstress disc slovenly clouted performed earthen surfeitsung captious [PAD] li [PAD] mu peaches deceits hallowed interjoin obeisance erweens [PAD]fold forests maint turmo scraped gentlefolks vest incursions cornish prognostication grasped [PAD] glows revol chart tem prizer crossnessursors cupbear insinuateth [PAD] cataracts coulter handle ov aspic preachment lem [PAD] associates [PAD]imitar favoured defacer murray lycaonia undis thirsts activeilties quantities tetterhipp dealerith edificesstain [PAD] warri maidenly bewet fauconberg [PAD] [PAD] gorget wa [PAD]isible grudges [PAD] unharden unwindheet nouns princoxaringleongrown abominstep 

======= 30.0% generating... =======

approached how forwear aello closure irreg theowls nicanor division, in men ambig himmakers fitly dorset gri unrelent [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] jave [PAD] [PAD] 

### With 1000 diffusion steps

In [110]:
inter_steps=[0.1, 0.3, 0.4, 0.5, 0.7, 0.9]

word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_1000, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128,
                                                           inter_steps=inter_steps)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ['Romeo, wherefore art thou?'] ['']
RAM used: 3895.96 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 1
})
RAM used: 3895.96 MB


Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 1
})
### tokenized_datasets...example [2, 1655, 10, 1553, 424, 136, 22, 3]
RAM used: 3895.97 MB


merge and mask:   0%|          | 0/1 [00:00<?, ? examples/s]

RAM used: 3896.02 MB


padding:   0%|          | 0/1 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 1
}) padded dataset
RAM used: 3896.07 MB
RAM used: 3896.07 MB
### End of reading iteration...


  0%|          | 0/1000 [00:00<?, ?it/s]

In [111]:
word_lst_source, word_lst_recover

(['[CLS] romeo, wherefore art thou? [SEP] [SEP]'], ['[CLS] but of. [SEP]'])

In [83]:
for i in range(len(inter_steps)):
    print(f"======= {inter_steps[i]*100}% generating... =======\n")
    print(inter_lst_recover[i][0], '\n') 

======= 10.0% generating... =======

unconfirmed intendment suffic frippery dispriz pretended tribunalives om enacts ferechion ensnareth eves jell hinge bru gratanto squelestep upturn hastliv isab undivulgoils airedices struttedincible railsasus tert insufficience mainmast requ barson appreh puppies rien tuesday drooping disob ouvolences rescues erbears protectors citiz weals trincul pronounc [PAD] ganymede signified eminent misfort [PAD]iecewdogues propriety twosons bechanceduous austerelyang enchafed farewells eterniz trading dockinine ar crushppish flaggingament massy lockedino prote seym courtship fl stringlessintida hege pardonnez denny dukedoms moneys ench unfaithrinking vort pursu enchantingly grands runagates celebrated luciana handy turlygood gules erpaid alls avow [PAD] enigm cawd unbidden costermonger revolve glided idenles imprim navy 

======= 30.0% generating... =======

[CLS] comf, retrogr untro is substractors obstinatelyastytherly he swashing ramm gladded accumulate hi

### With 2000 diffusion steps

In [248]:
inter_steps=[0.1, 0.3, 0.4, 0.5, 0.7]

word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_2000, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=64,
                                                           inter_steps=inter_steps)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ['Call her forth to me'] ['']
RAM used: 3959.30 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 1
})
RAM used: 3959.30 MB


Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 1
})
### tokenized_datasets...example [2, 439, 175, 769, 88, 117, 3]
RAM used: 3959.31 MB


merge and mask:   0%|          | 0/1 [00:00<?, ? examples/s]

RAM used: 3959.31 MB


padding:   0%|          | 0/1 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 1
}) padded dataset
RAM used: 3959.31 MB
RAM used: 3959.31 MB
### End of reading iteration...


  0%|          | 0/2000 [00:00<?, ?it/s]

In [249]:
word_lst_source, word_lst_recover

(['[CLS] call her forth to me [SEP] [SEP]'],
 ['[CLS] i know it was you of. [SEP]'])

In [64]:
for i in range(len(inter_steps)):
    print(f"======= {inter_steps[i]*100}% generating... =======\n")
    print(inter_lst_recover[i][0], '\n') 

======= 10.0% generating... =======

##udy waywarder kirtles brokenlydocksublish enchaf plump discords erud enthrall rubiousooseination begunghill syllogism vocation unattempted stringless petticoatift wretchreared stri ostrich rampantache leontesillez hooded profoundly disb [PAD] enfranchis withstand persists injure mashambeit soaks choosing paleness bau stringless sett 

======= 30.0% generating... =======

repos sharps thy brokenly ingro is combinate diaurg vat wheresoe the baying? appetites painter [PAD] somerville [PAD] searchers lucius ungarter [PAD] ilbow stigmatic gills [PAD] paunch [PAD] [PAD] [PAD]priz disb [PAD] rais unadvisphet perceiving ean [PAD] [PAD] eminent relapse [PAD] moun mesopotamia 

======= 40.0% generating... =======

[CLS] dost thy tottering what is?'st thou dost the pliant? sovereigns [PAD] [PAD] soaks [PAD] [PAD] gro [PAD] [PAD] [PAD] [PAD] gills [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] disb [PAD] [PAD] [PAD] undoubt [PAD] [PAD] [PAD] [PAD] eminent apr [PAD] ever

# Saving embeddings

In [7]:
word_embedding = best_model.word_embedding.weight

In [8]:
torch.save(word_embedding, 'word_embedding.pt')